In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd

from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_image
from functools import partial
from rasterio.enums import MergeAlg

import glob
import os

# Load Data

## Rent controlled and stabilized buildings

In [51]:
file_list = glob.glob(os.path.join("../data/L1/geocoded/", "*.csv"))

file_list
rent_stabilized_L = [pd.read_csv(file, sep = "\t") for file in file_list]
rent_stabilized = (pd.concat(rent_stabilized_L)
                   .query("ZIP!='ZIP'&STATUS1!='MULTIPLE DWELLING B'")
                  .loc[:,["loc", "lat", "lon"]]
                   .drop_duplicates("loc")
                   .dropna(subset = ['lat', 'lon'])
                  )


# assign each row a value of one to indicate the num of rent controlled locs (per row) for point density below
rent_stabilized['value'] = 1

rs_gpd = gpd.GeoDataFrame(rent_stabilized, geometry = gpd.points_from_xy(rent_stabilized.lon, rent_stabilized.lat), crs=4326)


# Transform to metered projection
# NAD83 (National Spatial Reference System 2011) Universal Transverse Mercator Zone 18 North projection
rs_gpd = rs_gpd.to_crs(6347)

In [52]:
type(rs_gpd)

geopandas.geodataframe.GeoDataFrame

In [53]:
# Create point density (sum of trees per 100m^2 grid cell)
pt_density = make_geocube(vector_data = rs_gpd,
                           measurements = ['value'],
                           resolution = (-100,100), # hectare resolution
                           rasterize_function = partial(rasterize_image, merge_alg = MergeAlg.add),
                           fill = 0) 

In [55]:
pt_density_ = pt_density/10000 # count / area

In [56]:
# Save rent stabilized point density
pt_density_.rio.to_raster('../data/L1/rent_stabilized_pointdens_ha.tiff')